In [49]:
import pandas as pd

In [92]:
# From GiantMIDI's github, they somehow chose tab as their delimiter instead of the traditional ","
#metadata = pd.read_csv("../full_music_pieces_youtube_similarity_pianosoloprob_split.csv", sep='\t')
metadata = pd.read_csv("../full_music_pieces_youtube_similarity_pianosoloprob.csv", sep='\t')
composers = pd.read_csv("../ComposerEraList.csv", usecols=['Full Name', 'Era'])

## Label Datapoint

In [51]:
composer2Era = dict(zip(composers["Full Name"], composers["Era"]))

In [52]:
# Helper function to do apply() on metadata dataframe
def findEra(x):
    era = 'unclear'
    name = x['firstname']+' '+x['surname']

    # Extract the era label from Composers.xlsx dataframe
    if name in composer2Era:
        era = composer2Era[name]
    return era

In [53]:
metadata['fullname'] = metadata.apply(lambda x: f"{x['firstname']} {x['surname']}", axis=1)

In [54]:
ds = metadata.loc[metadata['fullname'].isin(composer2Era.keys())]
ds['era'] = ds.apply(findEra, axis=1)

/var/folders/4r/z4c5m78x2tqgbkj8hx9jx2cm0000gn/T/ipykernel_7856/2693172142.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['era'] = ds.apply(findEra, axis=1)


In [56]:
ds.to_csv("../ds_labeled.csv")

## Extract MIDI Files

In [67]:
import os

In [101]:
ds = pd.read_csv("../ds_labeled.csv")
midi_path = "/Users/kinchang/GiantMIDI-Piano/workspace/midis"

In [102]:
ds_with_audio_name = ds[ds['audio_name'].notnull()]

In [104]:
found_midi = []
missing_midi = []
for name in ds_with_audio_name["audio_name"]:
    path = f"{midi_path}/{name}.mid"
    if os.path.isfile(path):
        found_midi.append(name)
    else:
        missing_midi.append(name)

In [105]:
print(f"found {len(found_midi)} of midis from dataset")
print(f"{len(missing_midi)} of music is missing")

found 948 of midis from dataset
5636 of music is missing


In [94]:
ds_with_midi = newDs[newDs['audio_name'].isin(found_midi)]

In [97]:
ds_with_midi.to_csv("../ds_labeled_with_midi.csv")

## Save MIDI Files

In [108]:
import shutil

In [109]:
src_folder = "/Users/kinchang/GiantMIDI-Piano/workspace/midis"
dst_folder = "/Users/kinchang/muzera/midis"

In [110]:
for name in found_midi:
    src = f"{src_folder}/{name}.mid"
    dst = f"{dst_folder}/{name}.mid"
    shutil.copy(src, dst)